
# Churn exploratory data analysis

**Notebook objectives**

- Check data quality
- Amend any data quality issues
- Identify trends or patterns that may help modelling
- Provide initial insights

In [5]:
# Import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

In [2]:
# Read in data
df =pd.read_csv('../01 Data/raw__churn_challenge__20210821.csv')

In [3]:
# Generate profile using pandas profiling
profile = ProfileReport(df, title="Churn Profiling Report", explorative = True)

In [4]:
# Display profile in notebook
profile.to_widgets()

Summarize dataset:   0%|          | 0/34 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
plt.bar(index, Rating, align='center', alpha=0.5)
plt.xticks(index, Movie_Name, rotation=30)
plt.ylabel('Rating')
plt.title('Movie Rating')
plt.show()

Text(0.5, 1.0, 'Target Variable "Churn"')

In [12]:
df.Churn

0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 7043, dtype: object